In [ ]:
!pip install torch torchvision


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transformations for the training and validation datasets
transform = transforms.Compose([
    transforms.Resize((64, 64)),   # Resize images to 64x64 pixels
    transforms.ToTensor(),         # Convert images to PyTorch tensors
    transforms.Normalize((0.5,), (0.5,))  # Normalize pixel values
])

# Define a custom function to ignore directories like .ipynb_checkpoints
def is_valid_file(filepath):
    return filepath.endswith(('.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff', '.webp'))

# Load the training and validation datasets
train_data = datasets.ImageFolder(root='/content/gdrive/MyDrive/Apple/train', transform=transform, is_valid_file=is_valid_file)
val_data = datasets.ImageFolder(root='/content/gdrive/MyDrive/Apple/test', transform=transform, is_valid_file=is_valid_file)

# Create data loaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

# Check the classes to ensure the dataset is loaded correctly
print(f"Classes: {train_data.classes}")



Classes: ['applesf', 'applesr']


In [ ]:
num_classes = len(train_data.classes)
print(f"Number of classes: {num_classes}")

Number of classes: 2


In [ ]:

class FruitClassifier(nn.Module):
    def __init__(self):
        super(FruitClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, 1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1)
        self.fc1 = nn.Linear(32 * 14 * 14, 128)
        self.fc2 = nn.Linear(128, num_classes)  # Adjusted to the number of classes

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2, 2)
        x = x.view(-1, 32 * 14 * 14)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x



In [ ]:
# Instantiate the model
model = FruitClassifier()

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
print(f"Classes: {train_data.classes}")
print(f"Number of classes: {len(train_data.classes)}")


Classes: ['applesf', 'applesr']
Number of classes: 2


In [ ]:
# Training loop
epochs = 30
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        print(f"Labels: {labels}")
        assert labels.max().item() < num_classes, "Label out of range!"

        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

    # Validation loop
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    print(f"Validation Loss: {val_loss/len(val_loader):.4f}")


Streaming output truncated to the last 5000 lines.
Labels: tensor([1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0,
        0, 1, 1, 1, 1, 0, 1, 0])
Labels: tensor([1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0,
        1, 1, 1, 1, 1, 1, 1, 1])
Labels: tensor([1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0,
        1, 0, 0, 0, 1, 1, 1, 1])
Labels: tensor([0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0,
        1, 0, 1, 1, 1, 1, 1, 1])
Labels: tensor([0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 0, 0, 1])
Labels: tensor([1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0,
        1, 0, 1, 0, 1, 0, 0, 0])
Labels: tensor([1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0,
        0, 0, 0, 0, 1, 1, 1, 1])
Labels: tensor([1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
        0, 0, 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive


'228W1A05C9 (1).pdf'
 228W1A05C9.pdf
 apple
 Apple2
 apple_classifier.pth
 banana
 banana_classifier.pth
 CCNA.pdf
 Classroom
'Colab Notebooks'
'College transcript.pdf'
'data science with Python.pdf'
 dataset
 data.yaml
 DOC-20240523-WA0060..pdf
'edu skills.pdf'
 ee.pdf
 Food-Spoilage-Detection-System
 fruit_classifier2.pth
 fruit_classifier3-2.pth
 fruit_classifier3.pth
 fruit_classifier_iot.pth
 fruit_classifier.pth
 fruit_video.mp4
 Harsha_Vedaraju.pdf
 IMG_20240217_121508.jpg
 orange
 orange_classifier.pth
 orng.ipynb
 RESUME1.pdf
 Screenshot_2023-12-19-16-49-21-29_40deb401b9ffe8e1df2f1cc5ba480b12.jpg
'Sree Harsha (1).pdf'
'Sree Harsha.pdf'
'Srija Yadagani.pdf'
 test123
'Uid  Model Question Paper.gdoc'
 VID_20240510215052.mp4
 VID-20240510-WA0071.mp4
 VID-20240510-WA0079.mp4
 workshopds
 yolov8n.pt


In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/apple_classifier.pth')
